<a href="https://colab.research.google.com/github/nishkalavallabhi/practicalnlp/blob/V_2_0/Ch4/fastText_demo%20(archived).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text classification dataset DBPedia. The dataset consists of text descriptions of 14 different classes. The training set contains 560,000 reviews and the test contains 70,000. Idea is to build a model that can predict the class from description.


Download dataset from [here](https://drive.google.com/drive/folders/0Bz8a_Dbh9Qhbfll6bVpmNUtUcFdjYmF2SEpmZUZUcVNiMUw1TWN6RDV3a0JHT3kxLVhVR2M). 



In [2]:
!pip install fasttext

     |████████████████████████████████| 61kB 4.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp27-cp27mu-linux_x86_64.whl size=2364880 sha256=d1ade8ce2a84d5c65a41ecdd1419bf6dd12e0b78373c0e32a7cd6715ceb16854
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [0]:
import os,sys  
import pandas as pd
import numpy as np
import fasttext

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [7]:
# Set dataset path

data_path = '/content/drive/NLP_book/Datasets/practicalnlp-master/Ch4/dbpedia_csv/dbpedia_csv/'

#Load train set
train_file = data_path + 'train.csv'
df = pd.read_csv(train_file, header=None, names=['class','name','description'])

#Load test set
test_file = data_path + 'test.csv'
df_test = pd.read_csv(test_file, header=None, names=['class','name','description'])

#Mapping from class number to class name
class_dict={
1:'Company',
2:'EducationalInstitution',
3:'Artist',
4:'Athlete',
5:'OfficeHolder',
6:'MeanOfTransportation',
7:'Building',
8:'NaturalPlace',
9:'Village',
10:'Animal',
11:'Plant',
12:'Album',
13:'Film',
14:'WrittenWork'
}
df['class_name'] = df['class'].map(class_dict)
df.head()
df.tail()

,class,name,description,class_name
559995,14,Barking in Essex,Barking in Essex is a Black comedy play direc...,WrittenWork
559996,14,Science & Spirit,Science & Spirit is a discontinued American b...,WrittenWork
559997,14,The Blithedale Romance,The Blithedale Romance (1852) is Nathaniel Ha...,WrittenWork
559998,14,Razadarit Ayedawbon,Razadarit Ayedawbon (Burmese: ရာဇာဓိရာဇ် အရေး...,WrittenWork
559999,14,The Vinyl Cafe Notebooks,Vinyl Cafe Notebooks: a collection of essays ...,WrittenWork


In [8]:
df.head()


,class,name,description,class_name
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...,Company
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...,Company
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...,Company
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...,Company
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...,Company


In [9]:

df.tail()

,class,name,description,class_name
559995,14,Barking in Essex,Barking in Essex is a Black comedy play direc...,WrittenWork
559996,14,Science & Spirit,Science & Spirit is a discontinued American b...,WrittenWork
559997,14,The Blithedale Romance,The Blithedale Romance (1852) is Nathaniel Ha...,WrittenWork
559998,14,Razadarit Ayedawbon,Razadarit Ayedawbon (Burmese: ရာဇာဓိရာဇ် အရေး...,WrittenWork
559999,14,The Vinyl Cafe Notebooks,Vinyl Cafe Notebooks: a collection of essays ...,WrittenWork


In [10]:
#df.describe().transpose()
desc = df.groupby('class')
desc.describe()

name         ...              class_name       
       count unique  ...                     top   freq
class                ...                               
1      40000  40000  ...                 Company  40000
2      40000  40000  ...  EducationalInstitution  40000
3      40000  40000  ...                  Artist  40000
4      40000  40000  ...                 Athlete  40000
5      40000  40000  ...            OfficeHolder  40000
6      40000  40000  ...    MeanOfTransportation  40000
7      40000  40000  ...                Building  40000
8      40000  40000  ...            NaturalPlace  40000
9      40000  40000  ...                 Village  40000
10     40000  40000  ...                  Animal  40000
11     40000  40000  ...                   Plant  40000
12     40000  40000  ...                   Album  40000
13     40000  40000  ...                    Film  40000
14     40000  40000  ...             WrittenWork  40000

[14 rows x 12 columns]

In [0]:
def clean_dataset(dataframe, shuffle=False, encode_ascii=False, clean_strings = False, label_prefix='__label__'):
    # Transform train file
    df = dataframe[['name','description']].apply(lambda x: x.str.replace(',',' '))
    df['class'] = label_prefix + dataframe['class'].astype(str) + ' '
    if clean_strings:
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('"',''))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('\'',' \' '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('.',' . '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('(',' ( '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace(')',' ) '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('!',' ! '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace('?',' ? '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace(':',' '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.replace(';',' '))
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.lower())
    if shuffle:
        df.sample(frac=1).reset_index(drop=True)
    if encode_ascii :
        df[['name','description']] = df[['name','description']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii','ignore').str.decode('utf-8'))
    df['name'] = ' ' + df['name'] + ' '
    df['description'] = ' ' + df['description'] + ' '
    return df

In [12]:
%%time
# Transform datasets
df_train_clean = clean_dataset(df, True, False)
df_test_clean = clean_dataset(df_test, False, False)

# Write files to disk
train_file_clean = data_path + 'dbpedia.train'
df_train_clean.to_csv(train_file_clean, header=None, index=False, columns=['class','name','description'] )

test_file_clean = data_path + 'dbpedia.test'
df_test_clean.to_csv(test_file_clean, header=None, index=False, columns=['class','name','description'] )

CPU times: user 15.5 s, sys: 10.7 s, total: 26.2 s
Wall time: 2min 20s


In [13]:
df_train_clean.head()

,name,description,class
0,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a B...,__label__1
1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for...,__label__1
2,Q-workshop,Q-workshop is a Polish company located in Po...,__label__1
3,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as R...,__label__1
4,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital lo...,__label__1


In [14]:
df_train_clean.tail()

,name,description,class
559995,Barking in Essex,Barking in Essex is a Black comedy play dire...,__label__14
559996,Science & Spirit,Science & Spirit is a discontinued American ...,__label__14
559997,The Blithedale Romance,The Blithedale Romance (1852) is Nathaniel H...,__label__14
559998,Razadarit Ayedawbon,Razadarit Ayedawbon (Burmese: ရာဇာဓိရာဇ် အရေ...,__label__14
559999,The Vinyl Cafe Notebooks,Vinyl Cafe Notebooks: a collection of essays...,__label__14


In [15]:
df['description'][10]

' Angstrem Group (Russian: \xd0\x9e\xd0\x90\xd0\x9e \xc2\xab\xd0\x90\xd0\xbd\xd0\xb3\xd1\x81\xd1\x82\xd1\x80\xd0\xb5\xd0\xbc\xc2\xbb named after angstrom) is a group of Russian companies one of the largest manufacturers of integrated circuits in Eastern Europe.The group includes: OAO Angstrem (the parent company design and manufacturing of electronic products and semiconductors); OAO Angstrem-M (custom design of integrated circuits staff training); OAO Angstrem-T (under-construction plant with 130-90 nm topology); OAO Angstrem-2M NGO Angstrem OAO Antek\xe2\x86\x91'

In [16]:
df_train_clean['description'][10]

'  Angstrem Group (Russian: \xd0\x9e\xd0\x90\xd0\x9e \xc2\xab\xd0\x90\xd0\xbd\xd0\xb3\xd1\x81\xd1\x82\xd1\x80\xd0\xb5\xd0\xbc\xc2\xbb named after angstrom) is a group of Russian companies one of the largest manufacturers of integrated circuits in Eastern Europe.The group includes: OAO Angstrem (the parent company design and manufacturing of electronic products and semiconductors); OAO Angstrem-M (custom design of integrated circuits staff training); OAO Angstrem-T (under-construction plant with 130-90 nm topology); OAO Angstrem-2M NGO Angstrem OAO Antek\xe2\x86\x91 '

In [18]:
%%time
# Train a classifier
output_file = data_path + 'dp_model'
classifier = fasttext.train_supervised(train_file_clean, output_file, label_prefix='__label__')

TypeError: ignored

In [0]:
%%time
# Evaluate classifier
result = classifier.test(test_file_clean)
print('P@1:', result.precision)
print('R@1:', result.recall)
print ('Number of examples:', result.nexamples)